# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [4]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Ask for parameters

In [15]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [16]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Indices:

*  $i$: Index for thermal units, $i = 1, 2, ..., 6$
* $t$: Index for time periods, $t = 1, 2, ..., 15$

## Parameters:

*  $SU_i$: Startup cost for unit $i$ (€) 
   * $SU_1 = 10324$, $SU_2 = 5678$, $SU_3 = 7802$, $SU_4 = 12899$, $SU_5 = 4596$, $SU_6 = 9076$
* $SD_i$: Shutdown cost for unit $i$ (€)
   *  $SD_1 = 2673$, $SD_2 = 5893$, $SD_3 = 982$, $SD_4 = 6783$, $SD_5 = 2596$, $SD_6 = 3561$
* $FC_i$: Fixed cost for unit $i$ (€)
* $VC_{i,t}$: Variable cost for unit $i$ at time $t$ (€/MWh)
* $P_{i}^{min}$: Minimum power output of unit $i$ (MW)
* $P_{i}^{max}$: Maximum power output of unit $i$ (MW)
* $RU_i$: Maximum power increment of unit $i$ from time $t$ to $t+1$ (MW)
* $RD_i$: Maximum power decrement of unit $i$ from time $t$ to $t+1$ (MW)
* $D_t$: Total power demand at time $t$ (MW)

## Variables:

* $P_{i,t}$: Power output of unit $i$ at time $t$ (MW)
* $U_{i,t}$: Binary variable indicating if unit $i$ is running at time $t$ (1 if on, 0 if off) 
* $SU_{i,t}$: Binary variable indicating if unit $i$ is started up at time $t$ (1 if started up, 0 otherwise)
* $SD_{i,t}$: Binary variable indicating if unit $i$ is shut down at time $t$ (1 if shut down, 0 otherwise) 


# 2. Ask for objective

In [17]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [18]:
Markdown(response2.text)

$$
\text{Minimize} \sum_{i=1}^{6} \sum_{t=1}^{15} (FC_i \cdot U_{i,t} + VC_{i,t} \cdot P_{i,t} + SU_i \cdot SU_{i,t} + SD_i \cdot SD_{i,t})
$$


# 3. Ask for constraints

In [19]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [20]:
Markdown(response3.text)

## Constraints:

**Power balance:**
$$\sum_{i=1}^{6} P_{i,t} \ge D_t, \quad \forall t = 1, 2, ..., 15$$

**Spinning reserve:**
$$\sum_{i=1}^{6} P_{i}^{max} \cdot U_{i,t}  \ge 1.1 \cdot D_t, \quad \forall t = 1, 2, ..., 15 $$

**Generation limits:**
$$P_{i}^{min} \cdot U_{i,t} \le P_{i,t} \le P_{i}^{max} \cdot U_{i,t}, \quad \forall i = 1, 2, ..., 6, \ \forall t = 1, 2, ..., 15$$

**Ramping limits:**
$$P_{i,t} - P_{i,t-1} \le RU_i,  \quad \forall i = 1, 2, ..., 6, \ \forall t = 2, 3, ..., 15$$
$$P_{i,t-1} - P_{i,t} \le RD_i, \quad \forall i = 1, 2, ..., 6, \ \forall t = 2, 3, ..., 15$$

**Startup and shutdown relationship:**
$$SU_{i,t} \ge U_{i,t} - U_{i,t-1}, \quad \forall i = 1, 2, ..., 6, \ \forall t = 2, 3, ..., 15$$
$$SD_{i,t} \ge U_{i,t-1} - U_{i,t}, \quad \forall i = 1, 2, ..., 6, \ \forall t = 2, 3, ..., 15$$

**Binary variables:**
$$U_{i,t}, \ SU_{i,t}, \ SD_{i,t} \in \{0,1\}, \quad \forall i = 1, 2, ..., 6, \ \forall t = 1, 2, ..., 15$$ 


## 3. Generate the pyomo code

In [21]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [22]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 110, 5: 70, 6: 95}  # Sample data
VC = {(i, t): 20 + i * t * 0.1 for i in range(1, 7) for t in range(1, 16)}  # Sample data
P_min = {1: 10, 2: 15, 3: 20, 4: 25, 5: 18, 6: 22}  # Sample data
P_max = {1: 100, 2: 80, 3: 90, 4: 150, 5: 70, 6: 120}  # Sample data
RU = {1: 30, 2: 25, 3: 35, 4: 40, 5: 28, 6: 32}  # Sample data
RD = {1: 30, 2: 25, 3: 35, 4: 40, 5: 28, 6: 32}  # Sample data
D = {t: 300 + t * 10 for t in range(1, 16)}  # Sample data

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, 6)  # Thermal units
model.T = pyo.RangeSet(1, 15)  # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.P_min = pyo.Param(model.I, initialize=P_min)
model.P_max = pyo.Param(model.I, initialize=P_max)
model.RU = pyo.Param(model.I, initialize=RU)
model.RD = pyo.Param(model.I, initialize=RD)
model.D = pyo.Param(model.T, initialize=D)

# Variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.U = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup variable
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown variable

# Objective function
def objective_rule(model):
    return sum(
        model.FC[i] * model.U[i, t]
        + model.VC[i, t] * model.P[i, t]
        + model.SU[i] * model.SU_var[i, t]
        + model.SD[i] * model.SD_var[i, t]
        for i in model.I
        for t in model.T
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

def spinning_reserve_rule(model, t):
    return sum(model.P_max[i] * model.U[i, t] for i in model.I) >= 1.1 * model.D[t]
model.spinning_reserve = pyo.Constraint(model.T, rule=spinning_reserve_rule)

def generation_limits_rule(model, i, t):
    return model.P_min[i] * model.U[i, t] <= model.P[i, t] <= model.P_max[i] * model.U[i, t]
model.generation_limits = pyo.Constraint(model.I, model.T, rule=generation_limits_rule)

def ramping_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t - 1] <= model.RU[i]
    else:
        return pyo.Constraint.Skip
model.ramping_up = pyo.Constraint(model.I, model.T, rule=ramping_up_rule)

def ramping_down_rule(model, i, t):
    if t > 1:
        return model.P[i, t - 1] - model.P[i, t] <= model.RD[i]
    else:
        return pyo.Constraint.Skip
model.ramping_down = pyo.Constraint(model.I, model.T, rule=ramping_down_rule)

def startup_relationship_rule(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip
model.startup_relationship = pyo.Constraint(model.I, model.T, rule=startup_relationship_rule)

def shutdown_relationship_rule(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown_relationship = pyo.Constraint(model.I, model.T, rule=shutdown_relationship_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')
solver.solve(model)

# Print the results
print("Optimal Solution:")
for t in model.T:
    print(f"Time period {t}:")
    for i in model.I:
        print(f"  Unit {i}: Power output = {pyo.value(model.P[i, t]):.2f} MW, On/Off = {pyo.value(model.U[i, t])}")
    print(f"  Total cost: {pyo.value(model.obj):.2f} €")
```

I have used sample data for:

*   `FC`: Fixed cost for each unit.
*   `VC`: Variable cost for each unit at each time period.
*   `P_min`: Minimum power output for each unit.
*   `P_max`: Maximum power output for each unit.
*   `RU`: Maximum ramp-up rate for each unit.
*   `RD`: Maximum ramp-down rate for each unit.
*   `D`: Total power demand for each time period. 


## 4. Input problem data and try running the generated code

In [23]:
import pyomo.environ as pyo

# Sample data (replace with actual data)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 100, 2: 80, 3: 90, 4: 110, 5: 70, 6: 95}  # Sample data
VC = {(i, t): 20 + i * t * 0.1 for i in range(1, 7) for t in range(1, 16)}  # Sample data
P_min = {1: 10, 2: 15, 3: 20, 4: 25, 5: 18, 6: 22}  # Sample data
P_max = {1: 100, 2: 80, 3: 90, 4: 150, 5: 70, 6: 120}  # Sample data
RU = {1: 30, 2: 25, 3: 35, 4: 40, 5: 28, 6: 32}  # Sample data
RD = {1: 30, 2: 25, 3: 35, 4: 40, 5: 28, 6: 32}  # Sample data
D = {t: 300 + t * 10 for t in range(1, 16)}  # Sample data

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, 6)  # Thermal units
model.T = pyo.RangeSet(1, 15)  # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.P_min = pyo.Param(model.I, initialize=P_min)
model.P_max = pyo.Param(model.I, initialize=P_max)
model.RU = pyo.Param(model.I, initialize=RU)
model.RD = pyo.Param(model.I, initialize=RD)
model.D = pyo.Param(model.T, initialize=D)

# Variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.U = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup variable
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown variable

# Objective function
def objective_rule(model):
    return sum(
        model.FC[i] * model.U[i, t]
        + model.VC[i, t] * model.P[i, t]
        + model.SU[i] * model.SU_var[i, t]
        + model.SD[i] * model.SD_var[i, t]
        for i in model.I
        for t in model.T
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

def spinning_reserve_rule(model, t):
    return sum(model.P_max[i] * model.U[i, t] for i in model.I) >= 1.1 * model.D[t]
model.spinning_reserve = pyo.Constraint(model.T, rule=spinning_reserve_rule)

def generation_limits_rule(model, i, t):
    return model.P_min[i] * model.U[i, t] <= model.P[i, t] <= model.P_max[i] * model.U[i, t]
model.generation_limits = pyo.Constraint(model.I, model.T, rule=generation_limits_rule)

def ramping_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t - 1] <= model.RU[i]
    else:
        return pyo.Constraint.Skip
model.ramping_up = pyo.Constraint(model.I, model.T, rule=ramping_up_rule)

def ramping_down_rule(model, i, t):
    if t > 1:
        return model.P[i, t - 1] - model.P[i, t] <= model.RD[i]
    else:
        return pyo.Constraint.Skip
model.ramping_down = pyo.Constraint(model.I, model.T, rule=ramping_down_rule)

def startup_relationship_rule(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip
model.startup_relationship = pyo.Constraint(model.I, model.T, rule=startup_relationship_rule)

def shutdown_relationship_rule(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown_relationship = pyo.Constraint(model.I, model.T, rule=shutdown_relationship_rule)

# Solve the model
solver = pyo.SolverFactory('cbc')
solver.solve(model)

# Print the results
print("Optimal Solution:")
for t in model.T:
    print(f"Time period {t}:")
    for i in model.I:
        print(f"  Unit {i}: Power output = {pyo.value(model.P[i, t]):.2f} MW, On/Off = {pyo.value(model.U[i, t])}")
    print(f"  Total cost: {pyo.value(model.obj):.2f} €")

ERROR: Rule failed when generating expression for Constraint generation_limits
with index (1, 1): PyomoException: Cannot convert non-constant Pyomo
expression (10*U[1,1]  <=  P[1,1]) to bool. This error is usually caused by
using a Var, unit, or mutable Param in a Boolean context such as an "if"
statement, or when checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'generation_limits' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression
        (10*U[1,1]  <=  P[1,1]) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ..

PyomoException: Cannot convert non-constant Pyomo expression (10*U[1,1]  <=  P[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [3]:
from pyomo.environ import *
import pyomo.environ as pyo


# Define sets
I = ['Plant1', 'Plant2', 'Plant3', 'Plant4', 'Plant5', 'Plant6']
T = range(1, 16)

# Sample data
# Startup costs
cu = {'Plant1': 10324, 'Plant2': 5678, 'Plant3': 7802, 'Plant4': 12899, 'Plant5': 4596, 'Plant6': 9076}
# Shutdown costs
cd = {'Plant1': 2673, 'Plant2': 5893, 'Plant3': 982, 'Plant4': 6783, 'Plant5': 2596, 'Plant6': 3561}
# Fixed costs
crf = {'Plant1': 2000, 'Plant2': 3000, 'Plant3': 2500, 'Plant4': 4000, 'Plant5': 3500, 'Plant6': 4500}
# Variable costs (changes for each time step)
crv = {
    'Plant1': [20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
    'Plant2': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    'Plant3': [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
    'Plant4': [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    'Plant5': [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
    'Plant6': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
}
# Lower and upper bounds for power output
l = {'Plant1': 50, 'Plant2': 40, 'Plant3': 30, 'Plant4': 60, 'Plant5': 55, 'Plant6': 65}
u = {'Plant1': 500, 'Plant2': 600, 'Plant3': 550, 'Plant4': 700, 'Plant5': 650, 'Plant6': 750}
# Ramp limits (up and down)
ramp_up = {'Plant1': 100, 'Plant2': 120, 'Plant3': 110, 'Plant4': 130, 'Plant5': 125, 'Plant6': 140}
ramp_down = {'Plant1': 90, 'Plant2': 110, 'Plant3': 100, 'Plant4': 120, 'Plant5': 115, 'Plant6': 130}
# Initial power output and running state
x0 = {'Plant1': 0, 'Plant2': 0, 'Plant3': 0, 'Plant4': 0, 'Plant5': 0, 'Plant6': 0}
v0 = {'Plant1': 0, 'Plant2': 0, 'Plant3': 0, 'Plant4': 0, 'Plant5': 0, 'Plant6': 0}
# Demand for each time period
d = {1: 280, 2: 327, 3: 400, 4: 388, 5: 501, 6: 600, 7: 800, 8: 927, 9: 705, 10: 502,
     11: 781, 12: 906, 13: 930, 14: 877, 15: 966}



# Sample data (replace with actual data)
SU =  {'Plant1': 10324, 'Plant2': 5678, 'Plant3': 7802, 'Plant4': 12899, 'Plant5': 4596, 'Plant6': 9076}
SD = {'Plant1': 2673, 'Plant2': 5893, 'Plant3': 982, 'Plant4': 6783, 'Plant5': 2596, 'Plant6': 3561}
FC = {'Plant1': 2000, 'Plant2': 3000, 'Plant3': 2500, 'Plant4': 4000, 'Plant5': 3500, 'Plant6': 4500}  # Sample data
VC = {
    ('Plant1', 1): 20, ('Plant1', 2): 22, ('Plant1', 3): 23, ('Plant1', 4): 24, ('Plant1', 5): 25, 
    ('Plant1', 6): 26, ('Plant1', 7): 27, ('Plant1', 8): 28, ('Plant1', 9): 29, ('Plant1', 10): 30, 
    ('Plant1', 11): 31, ('Plant1', 12): 32, ('Plant1', 13): 33, ('Plant1', 14): 34, ('Plant1', 15): 35,

    ('Plant2', 1): 15, ('Plant2', 2): 16, ('Plant2', 3): 17, ('Plant2', 4): 18, ('Plant2', 5): 19, 
    ('Plant2', 6): 20, ('Plant2', 7): 21, ('Plant2', 8): 22, ('Plant2', 9): 23, ('Plant2', 10): 24, 
    ('Plant2', 11): 25, ('Plant2', 12): 26, ('Plant2', 13): 27, ('Plant2', 14): 28, ('Plant2', 15): 29,

    ('Plant3', 1): 18, ('Plant3', 2): 19, ('Plant3', 3): 20, ('Plant3', 4): 21, ('Plant3', 5): 22, 
    ('Plant3', 6): 23, ('Plant3', 7): 24, ('Plant3', 8): 25, ('Plant3', 9): 26, ('Plant3', 10): 27, 
    ('Plant3', 11): 28, ('Plant3', 12): 29, ('Plant3', 13): 30, ('Plant3', 14): 31, ('Plant3', 15): 32,

    ('Plant4', 1): 25, ('Plant4', 2): 26, ('Plant4', 3): 27, ('Plant4', 4): 28, ('Plant4', 5): 29, 
    ('Plant4', 6): 30, ('Plant4', 7): 31, ('Plant4', 8): 32, ('Plant4', 9): 33, ('Plant4', 10): 34, 
    ('Plant4', 11): 35, ('Plant4', 12): 36, ('Plant4', 13): 37, ('Plant4', 14): 38, ('Plant4', 15): 39,

    ('Plant5', 1): 22, ('Plant5', 2): 23, ('Plant5', 3): 24, ('Plant5', 4): 25, ('Plant5', 5): 26, 
    ('Plant5', 6): 27, ('Plant5', 7): 28, ('Plant5', 8): 29, ('Plant5', 9): 30, ('Plant5', 10): 31, 
    ('Plant5', 11): 32, ('Plant5', 12): 33, ('Plant5', 13): 34, ('Plant5', 14): 35, ('Plant5', 15): 36,

    ('Plant6', 1): 30, ('Plant6', 2): 31, ('Plant6', 3): 32, ('Plant6', 4): 33, ('Plant6', 5): 34, 
    ('Plant6', 6): 35, ('Plant6', 7): 36, ('Plant6', 8): 37, ('Plant6', 9): 38, ('Plant6', 10): 39, 
    ('Plant6', 11): 40, ('Plant6', 12): 41, ('Plant6', 13): 42, ('Plant6', 14): 43, ('Plant6', 15): 44
}
 # Sample data
P_min = {'Plant1': 50, 'Plant2': 40, 'Plant3': 30, 'Plant4': 60, 'Plant5': 55, 'Plant6': 65}  # Sample data
P_max = {'Plant1': 500, 'Plant2': 600, 'Plant3': 550, 'Plant4': 700, 'Plant5': 650, 'Plant6': 750}  # Sample data
RU = {'Plant1': 100, 'Plant2': 120, 'Plant3': 110, 'Plant4': 130, 'Plant5': 125, 'Plant6': 140}  # Sample data
RD = {'Plant1': 90, 'Plant2': 110, 'Plant3': 100, 'Plant4': 120, 'Plant5': 115, 'Plant6': 130}  # Sample data
D =  {1: 280, 2: 327, 3: 400, 4: 388, 5: 501, 6: 600, 7: 800, 8: 927, 9: 705, 10: 502,
     11: 781, 12: 906, 13: 930, 14: 877, 15: 966}  # Sample data

# Model
model = pyo.ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.T = Set(initialize=T)  # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.P_min = pyo.Param(model.I, initialize=P_min)
model.P_max = pyo.Param(model.I, initialize=P_max)
model.RU = pyo.Param(model.I, initialize=RU)
model.RD = pyo.Param(model.I, initialize=RD)
model.D = pyo.Param(model.T, initialize=D)

# Variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.U = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment
model.SU_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup variable
model.SD_var = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown variable

# Objective function
def objective_rule(model):
    return sum(
        model.FC[i] * model.U[i, t]
        + model.VC[i, t] * model.P[i, t]
        + model.SU[i] * model.SU_var[i, t]
        + model.SD[i] * model.SD_var[i, t]
        for i in model.I
        for t in model.T
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) == model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

def spinning_reserve_rule(model, t):
    return sum(model.P_max[i] * model.U[i, t] for i in model.I) >= 1.1 * model.D[t]
model.spinning_reserve = pyo.Constraint(model.T, rule=spinning_reserve_rule)

def generation_limits_rule(model, i, t):
    return model.P_min[i] * model.U[i, t] <= model.P[i, t]
model.generation_limits = pyo.Constraint(model.I, model.T, rule=generation_limits_rule)

def generation_limits_ruleU(model, i, t):
    return  model.P[i, t] <= model.P_max[i] * model.U[i, t]
model.generation_limitsU = pyo.Constraint(model.I, model.T, rule=generation_limits_ruleU)

def ramping_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t - 1] <= model.RU[i]
    else:
        return model.P[i, t] <= model.RU[i]
model.ramping_up = pyo.Constraint(model.I, model.T, rule=ramping_up_rule)

def ramping_down_rule(model, i, t):
    if t > 1:
        return model.P[i, t - 1] - model.P[i, t] <= model.RD[i]
    else:
        return  - model.P[i, t] <= model.RD[i]
model.ramping_down = pyo.Constraint(model.I, model.T, rule=ramping_down_rule)

def startup_relationship_rule(model, i, t):
    if t > 1:
        return model.SU_var[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return model.SU_var[i, t] >= model.U[i, t]
model.startup_relationship = pyo.Constraint(model.I, model.T, rule=startup_relationship_rule)

def shutdown_relationship_rule(model, i, t):
    if t > 1:
        return model.SD_var[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return model.SD_var[i, t] >= - model.U[i, t]
model.shutdown_relationship = pyo.Constraint(model.I, model.T, rule=shutdown_relationship_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Solution:")
for t in model.T:
    print(f"Time period {t}:")
    for i in model.I:
        print(f"  Unit {i}: Power output = {pyo.value(model.P[i, t]):.2f} MW, On/Off = {pyo.value(model.U[i, t])}")
    print(f"  Total cost: {pyo.value(model.obj):.2f} €")

Optimal Solution:
Time period 1:
  Unit Plant1: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant2: Power output = 120.00 MW, On/Off = 1.0
  Unit Plant3: Power output = 105.00 MW, On/Off = 1.0
  Unit Plant4: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant5: Power output = 55.00 MW, On/Off = 1.0
  Unit Plant6: Power output = 0.00 MW, On/Off = 0.0
  Total cost: 367946.00 €
Time period 2:
  Unit Plant1: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant2: Power output = 240.00 MW, On/Off = 1.0
  Unit Plant3: Power output = 87.00 MW, On/Off = 1.0
  Unit Plant4: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant5: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant6: Power output = 0.00 MW, On/Off = 0.0
  Total cost: 367946.00 €
Time period 3:
  Unit Plant1: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant2: Power output = 360.00 MW, On/Off = 1.0
  Unit Plant3: Power output = 40.00 MW, On/Off = 1.0
  Unit Plant4: Power output = 0.00 MW, On/Off = 0.0
  Unit Plant5: Power output = 0.00 MW, On/

## 6. Print the responses

In [750]:
print(response.text)

## Variables:

**Production Quantities:**

*  $x_1$: Number of units of product P1 produced and sold weekly.
*  $x_2$: Number of units of product P2 produced and sold weekly.
*  $x_3$: Number of units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  $s_1$: Number of units of product P1 subcontracted for smelting weekly.
*  $s_2$: Number of units of product P2 subcontracted for smelting weekly. 



In [751]:
print(response2.text)

$$\max  \  1.2x_1 - 0.2s_1 + 1.3x_2 - 0.1s_2 + 1.57x_3$$ 



In [752]:
print(response3.text)

**Constraints:**

* **Production Capacity:**
    * Smelting: $6(x_1 - s_1) + 10(x_2 - s_2) + 8x_3 \le 8000$ 
    * Mechanization: $6x_1 + 3x_2 + 8x_3 \le 12000$
    * Assembly and Packaging: $3x_1 + 2x_2 + 2x_3 \le 10000$

* **Subcontracting Limits:**
    *  $s_1 \le x_1$ (Can't subcontract more than what's produced)
    *  $s_2 \le x_2$ (Can't subcontract more than what's produced)

* **Non-Negativity:**
    *  $x_1, x_2, x_3, s_1, s_2 \ge 0$ (Can't produce or subcontract negative quantities) 



In [753]:
print(response4.text)

```python
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Define variables
model.x1 = Var(domain=NonNegativeReals)  # Product P1 produced and sold
model.x2 = Var(domain=NonNegativeReals)  # Product P2 produced and sold
model.x3 = Var(domain=NonNegativeReals)  # Product P3 produced and sold
model.s1 = Var(domain=NonNegativeReals)  # Product P1 subcontracted
model.s2 = Var(domain=NonNegativeReals)  # Product P2 subcontracted

# Define the objective function
model.profit = Objective(
    expr=1.2 * model.x1 - 0.2 * model.s1 + 1.3 * model.x2 - 0.1 * model.s2 + 1.57 * model.x3,
    sense=maximize
)

# Define constraints
model.smelting = Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization = Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_packaging = Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1 = 